This notebook demonstrates the process of adding custom layers to the CoreML model during conversion. We discuss  three examples.

For TensorFlow operations (ops for short) that are not translatable to any of the CoreML layers, custom layers can be inserted in the CoreML model (list of CoreML layers can be found [here](https://github.com/apple/coremltools/blob/master/mlmodel/format/NeuralNetwork.proto) or [here](https://apple.github.io/coremltools/coremlspecification/sections/NeuralNetwork.html)). At runtime, CoreML framework will look for the implementation code of the custom layers, which has to be provided by the developer in her app.   
Custom layer is a [proto message](https://github.com/apple/coremltools/blob/5b5b8190764ffe78110be6b4d0edbeebe0253a6e/mlmodel/format/NeuralNetwork.proto#L2280), like any other neural network layer in the .mlmodel file (which is in the protobuf format), that can hold the parameters and weights (if any) associated with the TF op.
Here is the [documentation](https://developer.apple.com/documentation/coreml/core_ml_api/creating_a_custom_layer) on CoreML custom layers and a nice detailed [blogpost](http://machinethink.net/blog/coreml-custom-layers/). 

There are two ways in which a custom layer can be added during conversion from TF:

1. Specify the argument "add_custom_layers=True" during conversion. This will automatically check for unsupported ops and insert a coreml custom layer message in place of that op. The message can be later edited, if required, to add/remove any parameters.            

2. Specify the arguments "add_custom_layers=True" and "custom_conversion_functions" to the converter. The second argument is a dictionary, with keys that are either op types or op names and values are user-defined function handles. The functions receive TensorFlow [op](https://github.com/tensorflow/tensorflow/blob/51ef16057b4625e0a3e2943a9f1bbf856cf098ca/tensorflow/python/framework/ops.py#L3707) object and the CoreML neural network [builder object](https://github.com/apple/coremltools/blob/5b5b8190764ffe78110be6b4d0edbeebe0253a6e/coremltools/models/neural_network.py#L34) and give the user full control on how to handle the TF op and which layers to add to the CoreML graph. When the key is an op type, the function is called whenever op of that type is encountered while traversing the TF graph. Operation names as keys are useful for targeting specific ops. 

Lets now dive into the examples to make this process clear. 

First up, setting up some utilities:

In [1]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
from tensorflow.python.tools.freeze_graph import freeze_graph
import numpy as np
import shutil
import tempfile
import os
import tfcoreml
import coremltools
from coremltools.proto import NeuralNetwork_pb2
import netron # we use netron: https://github.com/lutzroeder/Netron for visualization. Comment out this line and all the calls to the "_visualize" method, if you do not want to use it. 

/Users/aseem/Aseem_env/env/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# A utility function to freeze rhe graph. It will be used later
def _simple_run_and_freeze(graph, output_name, frozen_model_file='', feed_dict={}):
    
    model_dir = tempfile.mkdtemp()
    graph_def_file = os.path.join(model_dir, 'tf_graph.pbtxt')
    checkpoint_file = os.path.join(model_dir, 'tf_model.ckpt')
    
    tf.reset_default_graph()
    with graph.as_default() as g:
      saver = tf.train.Saver()

    with tf.Session(graph = graph) as sess:
      # initialize
      sess.run(tf.global_variables_initializer())
      # run the result
      fetch = graph.get_operation_by_name(output_name).outputs[0]
      tf_result = sess.run(fetch, feed_dict=feed_dict)
      # save graph definition somewhere
      tf.train.write_graph(sess.graph, model_dir, graph_def_file)
      # save the weights
      saver.save(sess, checkpoint_file)
    
    freeze_graph(input_graph=graph_def_file,
                 input_saver="",
                 input_binary=False,
                 input_checkpoint=checkpoint_file,
                 output_node_names=output_name,
                 restore_op_name="save/restore_all",
                 filename_tensor_name="save/Const:0",
                 output_graph=frozen_model_file,
                 clear_devices=True,
                 initializer_nodes="")
    
    if os.path.exists(model_dir):
        shutil.rmtree(model_dir)
    
    return tf_result

# A utility function that takes an MLModel instance and prints info about Neural network layers inside.
# It prints short info about all the NN layers and the full description of any custom layer found
def _print_coreml_nn_layer_info(spec):
    nn_layers = coremltools.models.utils._get_nn_layers(spec)
    for i, layer in enumerate(nn_layers):
        if layer.WhichOneof('layer') == 'custom':
            print 'layer_id = ', i
            print layer
        else:
            print('{}: layer type: ({}) , inputs: {}, outputs: {}'.
              format(i,layer.WhichOneof('layer'), ", ".join([x for x in layer.input]), ", ".join([x for x in layer.output])))

# We use "netron" for visualization. 
def _visualize(network_path, port_number):
    
    def visualize_using_netron(path, port_number):
        netron.serve_file(path, browse = True, port=port_number)
    
    from threading import Thread
    import time
    
    d = Thread(target = visualize_using_netron, args = (network_path, port_number,))
    d.setDaemon(True)
    d.start()
    time.sleep(5)
    

Lets define the first TF graph. This one applies a dense layer and normalizes it. It uses the ["Tile"](https://www.tensorflow.org/versions/master/api_docs/python/tf/tile) op that CoreML does not support. 

In [3]:
# define a TF graph: input -> Dense -> unit norm -> output
graph = tf.Graph()
with graph.as_default() as g:
    inputs = tf.placeholder(tf.float32, shape=[None,8], name='input')
    with slim.arg_scope([slim.fully_connected],
          weights_initializer=tf.truncated_normal_initializer(0.0, 0.2),
          weights_regularizer=slim.l2_regularizer(0.0005)):
        y = slim.fully_connected(inputs, 10, scope='fc')
        y = slim.unit_norm(y,dim=1)

output_name = y.op.name
X = np.random.rand(1,8)
frozen_model_file = 'unit_norm_graph.pb'
coreml_model_path = 'unit_norm_graph.mlmodel'
out = _simple_run_and_freeze(graph, output_name, frozen_model_file, feed_dict={'input:0' : X})
print 'TF out: ', output_name, out.shape, np.sum(out ** 2)

INFO:tensorflow:Restoring parameters from /var/folders/09/hn9tml7d58nggsyxt54ymw5h0000gp/T/tmpp7o1Gk/tf_model.ckpt


INFO:tensorflow:Restoring parameters from /var/folders/09/hn9tml7d58nggsyxt54ymw5h0000gp/T/tmpp7o1Gk/tf_model.ckpt


INFO:tensorflow:Froze 2 variables.


INFO:tensorflow:Froze 2 variables.


Converted 2 variables to const ops.
TF out:  UnitNorm/div (1, 10) 0.9999999


In [4]:
# visualize the frozen TF model
_visualize(frozen_model_file, np.random.randint(8000, 9000))

Reading 'unit_norm_graph.pb'
Serving 'unit_norm_graph.pb' at http://localhost:8563


In [5]:
# Try to convert it : this call should raise an error
coreml_model = tfcoreml.convert(
        tf_model_path=frozen_model_file,
        mlmodel_path=coreml_model_path,
        input_name_shape_dict={'input:0':[1,8]},
        output_feature_names=['UnitNorm/div:0'])

Shapes not found for 10 tensors. Executing graph to determine shapes. 


NotImplementedError: Unsupported Ops of type: Tile

In [6]:
# we got an unsupported op error. Try again with custom Flag set to true
coreml_model = tfcoreml.convert(
        tf_model_path=frozen_model_file,
        mlmodel_path=coreml_model_path,
        input_name_shape_dict={'input:0':[1,8]},
        output_feature_names=['UnitNorm/div:0'],
        add_custom_layers=True)

Shapes not found for 10 tensors. Executing graph to determine shapes. 
1/26: Analysing op name: UnitNorm/concat/axis ( type:  Const )
2/26: Analysing op name: UnitNorm/StridedSlice/end ( type:  Const )
3/26: Analysing op name: UnitNorm/StridedSlice/begin ( type:  Const )
4/26: Analysing op name: UnitNorm/ones_like/Const ( type:  Const )
5/26: Analysing op name: UnitNorm/ones_like/Shape ( type:  Const )
6/26: Analysing op name: UnitNorm/ones_like ( type:  Fill )
7/26: Analysing op name: UnitNorm/ones ( type:  Const )
8/26: Analysing op name: UnitNorm/add/x ( type:  Const )
9/26: Analysing op name: UnitNorm/Sum/reduction_indices ( type:  Const )
10/26: Analysing op name: fc/biases ( type:  Const )
11/26: Analysing op name: fc/biases/read ( type:  Identity )
12/26: Analysing op name: fc/weights ( type:  Const )
13/26: Analysing op name: fc/weights/read ( type:  Identity )
14/26: Analysing op name: input ( type:  Placeholder )
Skipping name of placeholder
15/26: Analysing op name: fc/MatMu

We can see that the "Tile" op was made into a custom layer in the CoreML model. This op takes in two inputs, it recasts the first one into the shape given by the second input (by repetition). Here is the [documentation](https://www.tensorflow.org/versions/master/api_docs/python/tf/tile).  

In [7]:
# visualize CoreML model
_visualize(coreml_model_path, np.random.randint(8000, 9000))

Reading 'unit_norm_graph.mlmodel'
Serving 'unit_norm_graph.mlmodel' at http://localhost:8181


Note: As we can see in the visualization, the tensors whose values do not change based on the graph inputs (potentially they depend on the shape of the input, which needs to be fixed during conversion) are converted to "load constant" layers in the CoreML graph. 

In [8]:
# inspect the CoreML model
spec = coreml_model.get_spec()
_print_coreml_nn_layer_info(spec)

0: layer type: (innerProduct) , inputs: input__0, outputs: fc/BiasAdd:0
1: layer type: (activation) , inputs: fc/BiasAdd:0, outputs: fc/Relu:0
2: layer type: (loadConstant) , inputs: , outputs: UnitNorm/StridedSlice:0
3: layer type: (loadConstant) , inputs: , outputs: UnitNorm/ones:0
4: layer type: (concat) , inputs: UnitNorm/ones:0, UnitNorm/StridedSlice:0, outputs: UnitNorm/concat:0
5: layer type: (multiply) , inputs: fc/Relu:0, fc/Relu:0, outputs: UnitNorm/Square:0
6: layer type: (reduce) , inputs: UnitNorm/Square:0, outputs: UnitNorm/Sum:0
7: layer type: (add) , inputs: UnitNorm/Sum:0, outputs: UnitNorm/add:0
8: layer type: (unary) , inputs: UnitNorm/add:0, outputs: UnitNorm/Sqrt:0
layer_id =  9
name: "UnitNorm/Tile"
input: "UnitNorm/Sqrt:0"
input: "UnitNorm/concat:0"
output: "UnitNorm/Tile:0"
custom {
  className: "Tile"
  description: "Custom layer that corresponds to the TensorFlow op Tile"
}

10: layer type: (unary) , inputs: UnitNorm/Tile:0, outputs: inversed_UnitNorm/Tile:0_U

"ClassName" is an important message: this is the name of the swift/objective-c class that needs to implemented in the Xcode app and will contain the actual code for running the layer.  
The "tile" op does not have any parameters, so there is no need to edit generated the coreml specification. Lets now convert a TF graph with the op ["TopKV2"](https://www.tensorflow.org/api_docs/python/tf/nn/top_k) that has parameters. 

In [9]:
# define a TF graph: input -> Dense -> softmax -> top_k -> output
tf.reset_default_graph()
graph = tf.Graph()
with graph.as_default() as g:
    x = tf.placeholder(tf.float32, shape=[None,8], name="input")
    y = tf.layers.dense(inputs=x, units=12, activation=tf.nn.relu)
    y = tf.nn.softmax(y, axis=1)
    y = tf.nn.top_k(y, k=3, sorted = False, name='output')
    
output_name = 'output'    
X = np.random.rand(1,8)
frozen_model_file = 'topk_graph.pb'
coreml_model_path = 'topk_graph.mlmodel'
out = _simple_run_and_freeze(graph, output_name, frozen_model_file, feed_dict={'input:0' : X})
print 'TF out: ', output_name, out.shape, out

INFO:tensorflow:Restoring parameters from /var/folders/09/hn9tml7d58nggsyxt54ymw5h0000gp/T/tmpEm066N/tf_model.ckpt


INFO:tensorflow:Restoring parameters from /var/folders/09/hn9tml7d58nggsyxt54ymw5h0000gp/T/tmpEm066N/tf_model.ckpt


INFO:tensorflow:Froze 2 variables.


INFO:tensorflow:Froze 2 variables.


Converted 2 variables to const ops.
TF out:  output (1, 3) [[0.09352796 0.12875162 0.1025504 ]]


In [10]:
# visualize the frozen TF model
_visualize(frozen_model_file, np.random.randint(8000, 9000))

Reading 'topk_graph.pb'
Serving 'topk_graph.pb' at http://localhost:8202


In [11]:
# Try to convert it : this call should raise an error
coreml_model = tfcoreml.convert(
        tf_model_path=frozen_model_file,
        mlmodel_path=coreml_model_path,
        input_name_shape_dict={'input:0':[1,8]},
        output_feature_names=['output:0'])

Shapes not found for 7 tensors. Executing graph to determine shapes. 


NotImplementedError: Unsupported Ops of type: TopKV2

In [13]:
# we got an unsupported op error. Try again with custom Flag set to true
coreml_model = tfcoreml.convert(
        tf_model_path=frozen_model_file,
        mlmodel_path=coreml_model_path,
        input_name_shape_dict={'input:0':[1,8]},
        output_feature_names=['output:0'],
        add_custom_layers=True)

Shapes not found for 7 tensors. Executing graph to determine shapes. 
1/11: Analysing op name: output/k ( type:  Const )
2/11: Analysing op name: dense/bias ( type:  Const )
3/11: Analysing op name: dense/bias/read ( type:  Identity )
4/11: Analysing op name: dense/kernel ( type:  Const )
5/11: Analysing op name: dense/kernel/read ( type:  Identity )
6/11: Analysing op name: input ( type:  Placeholder )
Skipping name of placeholder
7/11: Analysing op name: dense/MatMul ( type:  MatMul )
8/11: Analysing op name: dense/BiasAdd ( type:  BiasAdd )
9/11: Analysing op name: dense/Relu ( type:  Relu )
10/11: Analysing op name: Softmax ( type:  Softmax )
11/11: Analysing op name: output ( type:  TopKV2 )
Adding custom layer

 Core ML model generated. Saved at location: topk_graph.mlmodel 

Core ML input(s): 
 [name: "input__0"
type {
  multiArrayType {
    shape: 8
    dataType: DOUBLE
  }
}
]
Core ML output(s): 
 [name: "output__0"
type {
  multiArrayType {
    shape: 3
    dataType: DOUBLE
 

In [14]:
# inspect the CoreML model
spec = coreml_model.get_spec()
_print_coreml_nn_layer_info(spec)

0: layer type: (innerProduct) , inputs: input__0, outputs: dense/BiasAdd:0
1: layer type: (activation) , inputs: dense/BiasAdd:0, outputs: dense/Relu:0
2: layer type: (softmax) , inputs: dense/Relu:0, outputs: Softmax:0
layer_id =  3
name: "output"
input: "Softmax:0"
input: "output/k:0"
output: "output__0"
output: "output:1"
custom {
  className: "TopKV2"
  description: "Custom layer that corresponds to the TensorFlow op TopKV2"
}



[top_k](https://www.tensorflow.org/api_docs/python/tf/nn/top_k) operation has two parameters: 'k' and 'sorted'. In the TF graph, the former is received as an additional input by the op and the latter is an op attribute. 
Let us modify the MLModel spec directly to add these two parameters to this layer. We need to know a little bit about the custom layer's [proto message](https://github.com/apple/coremltools/blob/5b5b8190764ffe78110be6b4d0edbeebe0253a6e/mlmodel/format/NeuralNetwork.proto#L2280) structure to be able to do that. 

In [15]:
nn_layers = coremltools.models.utils._get_nn_layers(spec) # get all the layers as a list
del nn_layers[3].input[1] # delete the second input: its just the value of k
del nn_layers[3].output[1] # delete the second output
nn_layers[3].custom.parameters["k"].intValue = 3
nn_layers[3].custom.parameters["sorted"].boolValue = False
_print_coreml_nn_layer_info(spec)

0: layer type: (innerProduct) , inputs: input__0, outputs: dense/BiasAdd:0
1: layer type: (activation) , inputs: dense/BiasAdd:0, outputs: dense/Relu:0
2: layer type: (softmax) , inputs: dense/Relu:0, outputs: Softmax:0
layer_id =  3
name: "output"
input: "Softmax:0"
output: "output__0"
custom {
  className: "TopKV2"
  parameters {
    key: "k"
    value {
      intValue: 3
    }
  }
  parameters {
    key: "sorted"
    value {
      boolValue: false
    }
  }
  description: "Custom layer that corresponds to the TensorFlow op TopKV2"
}



In [16]:
# save the spec back out
coremltools.models.utils.save_spec(spec, coreml_model_path)

In [17]:
# visualize CoreML model
_visualize(coreml_model_path, np.random.randint(8000, 9000))

Reading 'topk_graph.mlmodel'
Serving 'topk_graph.mlmodel' at http://localhost:8950


Here is an alternative way to do the same thing using the "custom_conversion_functions" argument: 

In [18]:
def _convert_topk(**kwargs):
    tf_op = kwargs["op"]
    coreml_nn_builder = kwargs["nn_builder"]
    constant_inputs = kwargs["constant_inputs"]
    
    params = NeuralNetwork_pb2.CustomLayerParams()
    params.className = 'Top_K'
    params.description = "Custom layer that corresponds to the top_k TF op"
    params.parameters["sorted"].boolValue = tf_op.get_attr('sorted')
    # get the value of k
    k = constant_inputs.get(tf_op.inputs[1].name, 3)
    params.parameters["k"].intValue = k
    coreml_nn_builder.add_custom(name=tf_op.name,
                                input_names=[tf_op.inputs[0].name],
                                output_names=[tf_op.outputs[0].name],
                                custom_proto_spec=params)

coreml_model = tfcoreml.convert(
        tf_model_path=frozen_model_file,
        mlmodel_path=coreml_model_path,
        input_name_shape_dict={'input:0':[1,8]},
        output_feature_names=['output:0'],
        add_custom_layers=True,
        custom_conversion_functions={'TopKV2': _convert_topk})

print("\n \n ML Model layers info: \n")
# inspect the CoreML model: this should be same as the one we got above
spec = coreml_model.get_spec()
_print_coreml_nn_layer_info(spec)

Shapes not found for 7 tensors. Executing graph to determine shapes. 
1/11: Analysing op name: output/k ( type:  Const )
2/11: Analysing op name: dense/bias ( type:  Const )
3/11: Analysing op name: dense/bias/read ( type:  Identity )
4/11: Analysing op name: dense/kernel ( type:  Const )
5/11: Analysing op name: dense/kernel/read ( type:  Identity )
6/11: Analysing op name: input ( type:  Placeholder )
Skipping name of placeholder
7/11: Analysing op name: dense/MatMul ( type:  MatMul )
8/11: Analysing op name: dense/BiasAdd ( type:  BiasAdd )
9/11: Analysing op name: dense/Relu ( type:  Relu )
10/11: Analysing op name: Softmax ( type:  Softmax )
11/11: Analysing op name: output ( type:  TopKV2 )
Adding custom layer

 Core ML model generated. Saved at location: topk_graph.mlmodel 

Core ML input(s): 
 [name: "input__0"
type {
  multiArrayType {
    shape: 8
    dataType: DOUBLE
  }
}
]
Core ML output(s): 
 [name: "output__0"
type {
  multiArrayType {
    shape: 3
    dataType: DOUBLE
 

Lets move on to the third and the final example. Now we will encounter an op that is supported but it errors out due to an unsupported coniguration. It is the [Slice](https://www.tensorflow.org/versions/master/api_docs/python/tf/slice) op.

In [19]:
# define a TF graph: input -> conv -> slice -> output
tf.reset_default_graph()
graph = tf.Graph()
with graph.as_default() as g:
    x = tf.placeholder(tf.float32, shape=[None,10,10,3], name="input")
    W = tf.Variable(tf.truncated_normal([1,1,3,5], stddev=0.1))
    y = tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
    y = tf.slice(y, begin=[0,1,1,1], size=[1,2,2,2], name='output')
    
output_name = 'output'    
X = np.random.rand(1,10,10,3)
frozen_model_file = 'slice_graph.pb'
coreml_model_path = 'slice_graph.mlmodel'
out = _simple_run_and_freeze(graph, output_name, frozen_model_file, feed_dict={'input:0' : X})
print 'TF out: ', output_name, out.shape

INFO:tensorflow:Restoring parameters from /var/folders/09/hn9tml7d58nggsyxt54ymw5h0000gp/T/tmpLTFwVp/tf_model.ckpt


INFO:tensorflow:Restoring parameters from /var/folders/09/hn9tml7d58nggsyxt54ymw5h0000gp/T/tmpLTFwVp/tf_model.ckpt


INFO:tensorflow:Froze 1 variables.


INFO:tensorflow:Froze 1 variables.


Converted 1 variables to const ops.
TF out:  output (1, 2, 2, 2)


In [20]:
# visualize the frozen TF model
_visualize(frozen_model_file, np.random.randint(8000, 9000))

Reading 'slice_graph.pb'
Serving 'slice_graph.pb' at http://localhost:8383


In [21]:
# Try to convert it : this call should raise an error
coreml_model = tfcoreml.convert(
        tf_model_path=frozen_model_file,
        mlmodel_path=coreml_model_path,
        input_name_shape_dict={'input:0':[1,10,10,3]},
        output_feature_names=['output:0'])

Shapes not found for 2 tensors. Executing graph to determine shapes. 
Automatic shape interpretation succeeded for input blob input:0
1/7: Analysing op name: output/size ( type:  Const )
2/7: Analysing op name: output/begin ( type:  Const )
3/7: Analysing op name: Variable ( type:  Const )
4/7: Analysing op name: Variable/read ( type:  Identity )
5/7: Analysing op name: input ( type:  Placeholder )
Skipping name of placeholder
6/7: Analysing op name: Conv2D ( type:  Conv2D )
7/7: Analysing op name: output ( type:  Slice )


NotImplementedError: Slice case not handled (input shape: [1, 10, 10, 5], output shape: [1, 2, 2, 2])

This fails, so we provide a custom layer function. Note that this time, the key in the dictionary provided via  "custom_conversion_functions" should be same as the op name ("output")

In [ ]:
def _convert_slice(**kwargs):
    tf_op = kwargs["op"]
    coreml_nn_builder = kwargs["nn_builder"]
    constant_inputs = kwargs["constant_inputs"]
    
    params = NeuralNetwork_pb2.CustomLayerParams()
    params.className = 'Slice'
    params.description = "Custom layer that corresponds to the slice TF op"
    # get the value of begin
    begin = constant_inputs.get(tf_op.inputs[1].name, [0,0,0,0])
    size = constant_inputs.get(tf_op.inputs[2].name, [0,0,0,0])
    # add begin and size as two repeated weight fields
    begin_as_weights = params.weights.add()
    begin_as_weights.floatValue.extend(map(float, begin))
    size_as_weights = params.weights.add()
    size_as_weights.floatValue.extend(map(float, size))
    coreml_nn_builder.add_custom(name=tf_op.name,
                                input_names=[tf_op.inputs[0].name],
                                output_names=[tf_op.outputs[0].name],
                                custom_proto_spec=params)

coreml_model = tfcoreml.convert(
        tf_model_path=frozen_model_file,
        mlmodel_path=coreml_model_path,
        input_name_shape_dict={'input:0':[1,10,10,3]},
        output_feature_names=['output:0'],
        add_custom_layers=True,
        custom_conversion_functions={'output': _convert_slice}) # dictionary has op name as the key

print("\n \n ML Model layers info: \n")
# inspect the CoreML model: this should be same as the one we got above
spec = coreml_model.get_spec()
_print_coreml_nn_layer_info(spec)

In [ ]:
# visualize CoreML model
_visualize(coreml_model_path, np.random.randint(8000, 9000))